In [ ]:
!pip install torch torchvision torchaudio
!pip install torch-geometric

In [23]:
import numpy as np
import time

In [13]:
import torch

from torch_geometric.datasets import Planetoid

dataset_name = 'Cora'

root = './data'
dataset = Planetoid(root=root, name=dataset_name)
data = dataset[0]

# Access nodes and edges
nodes = data.x  # Node features
edges = data.edge_index  # Edges

Processing...
Done!


The `sanityCheck function` takes a file name and format as input, reads the file, and counts the number of nodes and edges in the graph. It handles both space-separated and comma-separated formats.

In [14]:
def sanityCheck(edge_index):
    nodes = int(edge_index.max()) + 1
    edges = edge_index.shape[1]
    return nodes, edges


The `initGraph function` initializes the graph by creating an adjacency list (adj) and an array of degrees (degrees) based on the input file. It then saves these arrays to files with specific names.

In [15]:
def initGraph(edge_index, nb_nodes):
    adj = np.empty(nb_nodes, dtype=object)
    degrees = np.zeros(nb_nodes, dtype=int)
    for i in np.ndindex(adj.shape):
        adj[i] = []
    for edge in edge_index.t().numpy():
        i, j = edge[0], edge[1]
        degrees[i] += 1
        degrees[j] += 1
        adj[i].append(j)
        adj[j].append(i)
    np.save('adj_cora', adj)
    np.save('degrees_cora', degrees)
    return

    The greedy function implements a greedy algorithm to find a densest subgraph. It takes the file name, the number of nodes, and the number of edges as inputs. It uses precomputed adjacency lists and degree arrays.
    
    The algorithm iteratively removes vertices with the lowest degree along with their incident edges until no vertices remain. It updates the density and the number of nodes in the densest subgraph during each iteration.

In [21]:
def greedy(nb_nodes, nb_edges):
    adj = np.load('adj_cora.npy', allow_pickle=True)
    degrees = np.load('degrees_cora.npy', allow_pickle=True)
    erased = np.zeros(nb_nodes, dtype=bool)
    vertices = np.empty(nb_nodes, dtype=object)
    for i in np.ndindex(vertices.shape):
        vertices[i] = []
    locations = np.zeros(nb_nodes, dtype=int)
    mind = nb_nodes
    for i in range(nb_nodes):
        vertices[degrees[i]].append(i)
        locations[i] = len(vertices[degrees[i]]) - 1
        if degrees[i] < mind:
            mind = degrees[i]
    n = nb_nodes
    m = nb_edges
    density = m / n
    subgraph_nodes = n

    while n != 0:
        while len(vertices[mind]) == 0 and mind < nb_nodes - 1:
            mind += 1

        if mind == nb_nodes - 1 and len(vertices[mind]) == 0:
            break  # All vertices have been removed

        imin = vertices[mind].pop()
        erased[imin] = 1
        neighbors = adj[imin]
        backwards = False
        for j in neighbors:
            if not erased[j]:
                m -= 1
                d = degrees[j]
                loc = locations[j]
                length = len(vertices[d])
                vertices[d][loc], vertices[d][length - 1] = vertices[d][length - 1], vertices[d][loc]
                locations[vertices[d][loc]] = loc
                vertices[d].pop()
                vertices[d - 1].append(j)
                locations[j] = len(vertices[d - 1]) - 1
                degrees[j] -= 1
                if d == mind:
                    backwards = True

        if backwards:
            mind -= 1

        n -= 1
        if n != 0:
            if density < m / n:
                density = m / n
                subgraph_nodes = n

    return density, subgraph_nodes


In [18]:
# Save the edges in txt format (you may need to adjust this based on the format)
edges_file = 'cora_edges.txt'
with open(edges_file, 'w') as f:
    for edge in data.edge_index.t().numpy():
        f.write(f"{edge[0]} {edge[1]}\n")

In [19]:
# Perform sanity check and initialize the graph
nodes, edges = sanityCheck(data.edge_index)
initGraph(data.edge_index, nodes)

In [24]:
# Run the greedy algorithm on the Cora dataset
begin = time.time()
density, subgraph_nodes = greedy(nodes, edges)
end = time.time()
print(f"Density: {density}, Subgraph Nodes: {subgraph_nodes}")
print("Found a result in", end - begin, "seconds")

Density: 4.497658862876254, Subgraph Nodes: 1495
Found a result in 0.09453034400939941 seconds
